In [18]:
#importar librerias 
import pandas as pd 
import numpy as np
import nltk 

In [121]:
#Cargar la data 
df_solicitudes = pd.read_csv("../files/input/solicitudes_de_credito.csv", sep = ";", index_col = 0)
df_solicitudes 

,sexo,tipo_de_emprendimiento,idea_negocio,barrio,estrato,comuna_ciudadano,fecha_de_beneficio,monto_del_credito,línea_credito
0,masculino,comercio,fabrica de,prado,3,10.0,13/07/2018,5000000,microempresarial
1,femenino,servicio,recreativo-educativo,barrio caicedo,1,9.0,30/10/2017,6000000,microempresarial
2,femenino,comercio,tienda,aranjuez,2,4.0,3/08/2017,7300000,microempresarial
3,Masculino,comercio,variedades,cabecera san cristobal,3,60.0,20/11/2017,7000000,microempresarial
4,femenino,comercio,fabrica de,robledo,2,7.0,23/02/2017,5000000,microempresarial
...,...,...,...,...,...,...,...,...,...
10915,masculino,agropecuaria,comidas rapidas,villatina,1,8.0,9/05/2018,7000000,microempresarial
10916,masculino,comercio,fabrica de arepas,villa hermosa,3,8.0,23/01/2017,7377000,microempresarial
10917,femenino,comercio,variedades,cabecera san cristobal,1,60.0,12/04/2018,7800000,microempresarial
10918,femenino,servicio,cafe e internet,manrique oriental,2,3.0,1/06/2017,7370000,microempresarial


In [ ]:
print("=" * 60)
print("PASO 1: EXPLORACIÓN INICIAL")
print("=" * 60)

print(f"\nDimensiones: {df_solicitudes.shape}")
print(f"\nPrimeras filas:")
print(df_solicitudes.head())
print(f"\nInformación general:")
print(df_solicitudes.info())
print(f"\nEstadísticas descriptivas:")
print(df_solicitudes.describe(include='all'))

print("\n" + "=" * 60)
print("PASO 2: DETECTAR VALORES FALTANTES")
print("=" * 60)

print(f"\nValores nulos por columna:")
print(df_solicitudes.isnull().sum())
print(f"\nPorcentaje de nulos:")
print((df_solicitudes.isnull().sum() / len(df_solicitudes) * 100).round(2))

# Visualizar patrones de nulos
print(f"\nTotal de filas con al menos un nulo: {df_solicitudes.isnull().any(axis=1).sum()}")


PASO 1: EXPLORACIÓN INICIAL

Dimensiones: (10920, 9)

Primeras filas:
        sexo tipo_de_emprendimiento          idea_negocio  \
0  masculino               comercio           fabrica de    
1   femenino               servicio  recreativo-educativo   
2   femenino               comercio                tienda   
3  Masculino               comercio            variedades   
4   femenino               comercio           fabrica de    

                   barrio  estrato  comuna_ciudadano fecha_de_beneficio  \
0                   prado        3              10.0         13/07/2018   
1          barrio caicedo        1               9.0         30/10/2017   
2                aranjuez        2               4.0          3/08/2017   
3  cabecera san cristobal        3              60.0         20/11/2017   
4                 robledo        2               7.0         23/02/2017   

  monto_del_credito     línea_credito  
0           5000000  microempresarial  
1           6000000  microempres

In [160]:
print("\n" + "=" * 60)
print("PASO 3: LIMPIEZA DE STRINGS")
print("=" * 60)

# Identificar columnas de texto
columnas_texto = df_solicitudes.select_dtypes(include=['object']).columns
print(columnas_texto)

for col in columnas_texto:
    print(f"\nLimpiando columna: {col}")
    
    # Eliminar espacios al inicio y final
    df_solicitudes[col] = df_solicitudes[col].str.strip()
    
    # Convertir a minúsculas (opcional)
    df_solicitudes[col] = df_solicitudes[col].str.lower()
    
    # Eliminar espacios dobles
    df_solicitudes[col] = df_solicitudes[col].str.replace(r'\s+', ' ', regex=True)
    
    # Reemplazar valores vacíos por NaN
    df_solicitudes[col] = df_solicitudes[col].replace(['', ' ', 'N/A', 'NA', 'null', 'None'], np.nan)
    
    # Detectar caracteres especiales problemáticos
    caracteres_raros = df_solicitudes[col].str.contains(r'[^\w\s\-.,áéíóúñÁÉÍÓÚÑ]', na=False, regex=True)
    if caracteres_raros.any():
        print(f"  ⚠️  Caracteres especiales encontrados: {caracteres_raros.sum()} casos")


PASO 3: LIMPIEZA DE STRINGS
Index(['sexo', 'tipo_de_emprendimiento', 'idea_negocio', 'barrio',
       'línea_credito'],
      dtype='object')

Limpiando columna: sexo

Limpiando columna: tipo_de_emprendimiento

Limpiando columna: idea_negocio

Limpiando columna: barrio
  ⚠️  Caracteres especiales encontrados: 8 casos

Limpiando columna: línea_credito


In [161]:
df_solicitudes["sexo"].value_counts().tolist()

[7078, 3842]

In [165]:
# Identificar columnas numéricas
columnas_numericas = df_solicitudes.select_dtypes(include=[np.number]).columns
print(columnas_numericas)
for col in columnas_numericas:
    print(f"\nAnalizando columna numérica: {col}")
    
    # Estadísticas
    print(f"  Min: {df_solicitudes[col].min()}, Max: {df_solicitudes[col].max()}")
    print(f"  Media: {df_solicitudes[col].mean():.2f}, Mediana: {df_solicitudes[col].median():.2f}")
    
    # Detectar valores atípicos (outliers) usando IQR
    Q1 = df_solicitudes[col].quantile(0.25)
    Q3 = df_solicitudes[col].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    outliers = ((df_solicitudes[col] < limite_inferior) | (df_solicitudes[col] > limite_superior))
    num_outliers = outliers.sum()
    
    if num_outliers > 0:
        print(f"  ⚠️  Outliers detectados: {num_outliers} ({num_outliers/len(df_solicitudes)*100:.2f}%)")
        print(f"  Rango esperado: [{limite_inferior:.2f}, {limite_superior:.2f}]")
    
    # Detectar valores negativos si no deberían existir
    negativos = (df_solicitudes[col] < 0).sum()
    if negativos > 0:
        print(f"  ⚠️  Valores negativos: {negativos}")
    
    # Detectar valores cero
    ceros = (df_solicitudes[col] == 0).sum()
    if ceros > 0:
        print(f"  ℹ️  Valores cero: {ceros}")

Index(['estrato', 'comuna_ciudadano', 'monto_del_credito'], dtype='object')

Analizando columna numérica: estrato
  Min: 0, Max: 3
  Media: 2.11, Mediana: 2.00
  ⚠️  Outliers detectados: 3 (0.03%)
  Rango esperado: [0.50, 4.50]
  ℹ️  Valores cero: 3

Analizando columna numérica: comuna_ciudadano
  Min: 1.0, Max: 90.0
  Media: 11.67, Mediana: 7.00
  ⚠️  Outliers detectados: 836 (7.66%)
  Rango esperado: [-5.00, 19.00]

Analizando columna numérica: monto_del_credito
  Min: 6.0, Max: 82000000.0
  Media: 6298068.86, Mediana: 6890000.00
  ⚠️  Outliers detectados: 210 (1.92%)
  Rango esperado: [-65500.00, 11842500.00]


In [166]:
df_solicitudes["sexo"].value_counts().tolist()

[7078, 3842]

In [167]:
# Convertir columnas que parecen numéricas pero están como texto
print("\nConvirtiendo columnas numéricas mal formateadas...")
for col in df_solicitudes.select_dtypes(include=['object']).columns:
    # Intentar convertir a numérico
    convertido = pd.to_numeric(df_solicitudes[col], errors='coerce')
    if convertido.notna().sum() > len(df_solicitudes) * 0.8:  # Si >80% son números
        print(f"  ✓ Convirtiendo '{col}' a numérico")
        df_solicitudes[col] = convertido



Convirtiendo columnas numéricas mal formateadas...


In [168]:
print("\n" + "=" * 60)
print("PASO 5: LIMPIEZA DE FECHAS")
print("=" * 60)

# Detectar y limpiar columnas de fechas
posibles_fechas = [col for col in df_solicitudes.columns if any(keyword in col.lower() 
                   for keyword in ['fecha', 'date', 'time', 'dia', 'mes', 'año'])]

for col in posibles_fechas:
    print(f"\nProcesando columna de fecha: {col}")
    
    try:
        # Intentar convertir a datetime
        df_solicitudes[col] = pd.to_datetime(df_solicitudes[col], errors='coerce')
        
        # Detectar fechas inválidas
        fechas_nulas = df_solicitudes[col].isnull().sum()
        if fechas_nulas > 0:
            print(f"  ⚠️  Fechas inválidas: {fechas_nulas}")
        
        # Detectar fechas futuras (si no deberían existir)
        fechas_futuras = (df_solicitudes[col] > pd.Timestamp.now()).sum()
        if fechas_futuras > 0:
            print(f"  ⚠️  Fechas futuras detectadas: {fechas_futuras}")
        
        # Rango de fechas
        if df_solicitudes[col].notna().any():
            print(f"  Rango: {df_solicitudes[col].min()} a {df_solicitudes[col].max()}")
            
    except Exception as e:
        print(f"  ✗ Error al convertir: {e}")


PASO 5: LIMPIEZA DE FECHAS

Procesando columna de fecha: fecha_de_beneficio
  Rango: 2016-01-05 00:00:00 a 2019-06-28 00:00:00


In [170]:
df_solicitudes["sexo"].value_counts().tolist()

[7078, 3842]

In [188]:
print("\n" + "=" * 60)
print("PASO 6: DETECTAR DUPLICADOS")
print("=" * 60)

duplicados = df_solicitudes.duplicated().sum()
print(f"\nFilas duplicadas completas: {duplicados}")

if duplicados > 0:
    print(f"Porcentaje: {duplicados/len(df_solicitudes)*100:.2f}%")
    df_solicitudes = df_solicitudes.drop_duplicates()  # Descomentar para eliminar


PASO 6: DETECTAR DUPLICADOS

Filas duplicadas completas: 0


In [189]:
df_solicitudes["sexo"].value_counts().tolist()

[6810, 3688]

In [190]:
print("\n" + "=" * 60)
print("PASO 7: TRATAR VALORES FALTANTES")
print("=" * 60)

print("\nOpciones para tratar valores faltantes:")

# Opción A: Eliminar filas con muchos nulos
umbral = 0.5  # Si >50% de columnas son nulas
filas_antes = len(df_solicitudes)
df_sin_nulos_excesivos = df_solicitudes.dropna(thresh=len(df_solicitudes.columns) * umbral)
print(f"A. Eliminando filas con >{umbral*100}% nulos: {filas_antes - len(df_sin_nulos_excesivos)} filas eliminadas")

# Opción B: Imputar valores numéricos
for col in columnas_numericas:
    if df_solicitudes[col].isnull().sum() > 0:
        # Con la mediana (más robusto a outliers)
        df_solicitudes[col].fillna(df_solicitudes[col].median(), inplace=True)
        
        # Con la media
        # df[col].fillna(df[col].mean(), inplace=True)
        
        # Con interpolación
        # df[col].interpolate(method='linear', inplace=True)
        pass

# Opción C: Imputar valores categóricos
for col in columnas_texto:
    if df_solicitudes[col].isnull().sum() > 0:
        # Con la moda (valor más frecuente)
        df_solicitudes[col].fillna(df_solicitudes[col].mode()[0], inplace=True)
        
        # Con un valor por defecto
        # df[col].fillna('Desconocido', inplace=True)
        pass

print("\n" + "=" * 60)
print("PASO 8: VALIDAR CONSISTENCIA")
print("=" * 60)

# Verificar rangos válidos
print("\nVerificando rangos de valores...")

# Ejemplo: edades entre 0 y 120
if 'edad' in df_solicitudes.columns:
    edades_invalidas = ((df_solicitudes['edad'] < 0) | (df_solicitudes['edad'] > 120)).sum()
    print(f"Edades inválidas: {edades_invalidas}")

# Verificar coherencia entre columnas
# Ejemplo: fecha_fin > fecha_inicio
if 'fecha_inicio' in df_solicitudes.columns and 'fecha_fin' in df_solicitudes.columns:
    incoherentes = (df_solicitudes['fecha_fin'] < df_solicitudes['fecha_inicio']).sum()
    print(f"Fechas incoherentes: {incoherentes}")


PASO 7: TRATAR VALORES FALTANTES

Opciones para tratar valores faltantes:
A. Eliminando filas con >50.0% nulos: 0 filas eliminadas

PASO 8: VALIDAR CONSISTENCIA

Verificando rangos de valores...


In [193]:
df_solicitudes["sexo"].value_counts().tolist()

[6810, 3688]

In [83]:
# Método 2: Filtrar solo columnas string/object
columnas_string = df_solicitudes.select_dtypes(include=['object']).columns.tolist()
print("Columnas de texto:", columnas_string)

columnas_numericas = df_solicitudes.select_dtypes(include=['int64', 'float64']).columns.tolist()
print("Columnas numéricas:", columnas_numericas)

Columnas de texto: ['sexo', 'tipo_de_emprendimiento', 'idea_negocio', 'barrio', 'fecha_de_beneficio', 'monto_del_credito', 'línea_credito']
Columnas numéricas: ['estrato', 'comuna_ciudadano']


In [84]:
df_solicitudes.columns
Lista_columnas = df_solicitudes.columns.tolist()
print(Lista_columnas)


['sexo', 'tipo_de_emprendimiento', 'idea_negocio', 'barrio', 'estrato', 'comuna_ciudadano', 'fecha_de_beneficio', 'monto_del_credito', 'línea_credito']


In [85]:
def generar_ngramas(texto, n=2):
    """Genera n-gramas a partir de un texto"""
    if pd.isna(texto):  # Verificar valores nulos
        return []
    texto = str(texto).replace(" ", "")
    if len(texto) < n:
        return []
    return [texto[t:t+n] for t in range(len(texto)-n+1)]

def similitud_ngramas(palabra1, palabra2, n=2):
    """Calcula similitud entre dos palabras usando n-gramas"""
    # Verificar valores nulos
    if pd.isna(palabra1) or pd.isna(palabra2):
        return 0
    
    ngramas1 = set(generar_ngramas(palabra1, n))
    ngramas2 = set(generar_ngramas(palabra2, n))
    
    if len(ngramas1) == 0 or len(ngramas2) == 0:
        return 0
    
    # Intersección / Unión (Jaccard)
    interseccion = len(ngramas1 & ngramas2)
    union = len(ngramas1 | ngramas2)
    
    return interseccion / union

def limpiar_variaciones(df, columna, umbral=0.8):
    """Reemplaza palabras similares por una versión canónica"""
    df_copia = df.copy()
    valores_unicos = df_copia[columna].dropna().unique()
    
    # Mapeo de variaciones
    mapeo = {}
    valores_list = list(valores_unicos)
    
    for i, valor in enumerate(valores_list):
        if valor in mapeo:
            continue
            
        # Buscar palabras similares
        for otro_valor in valores_list[i+1:]:
            if otro_valor not in mapeo:
                similitud = similitud_ngramas(valor, otro_valor)
                
                if similitud >= umbral:
                    # Usar la más corta como canónica
                    canonical = valor if len(str(valor)) <= len(str(otro_valor)) else otro_valor
                    mapeo[otro_valor] = canonical
    
    # Aplicar mapeo
    df_copia[columna] = df_copia[columna].map(lambda x: mapeo.get(x, x) if pd.notna(x) else x)
    
    return df_copia

In [194]:
# Primero instalar: pip install fuzzywuzzy python-Levenshtein
from fuzzywuzzy import fuzz

def limpiar_fuzzy(df, columna, umbral=85):
    """Usa fuzzy matching para encontrar similitudes"""
    df_copia = df.copy()
    valores_unicos = df_copia[columna].dropna().unique()
    
    mapeo = {}
    valores_list = list(valores_unicos)
    
    for i, valor in enumerate(valores_list):
        if valor in mapeo:
            continue
            
        for otro_valor in valores_list[i+1:]:
            if otro_valor not in mapeo:
                # fuzz.ratio retorna 0-100
                similitud = fuzz.ratio(valor.lower(), otro_valor.lower())
                
                if similitud >= umbral:
                    canonical = valor if len(valor) <= len(otro_valor) else otro_valor
                    mapeo[otro_valor] = canonical
    
    df_copia[columna] = df_copia[columna].map(lambda x: mapeo.get(x, x) if pd.notna(x) else x)
    return df_copia

# Uso

df_final = limpiar_fuzzy(df_solicitudes, columna='sexo', umbral=99)

In [207]:
df_final["sexo"].value_counts().tolist()

[6810, 3688]

In [213]:
def limpiar_dataset_completo(df, umbral_similitud=0.36):
    df_copia = df.copy()
    
    # Paso 1: Normalizar texto
    columnas_string = df_copia.select_dtypes(include=['object']).columns
    
    for columna in columnas_string:
        # Llenar NaN temporalmente
        df_copia[columna] = df_copia[columna].fillna("desconocido")
        df_copia[columna] = df_copia[columna].astype(str)
        df_copia[columna] = df_copia[columna].str.strip()
        df_copia[columna] = df_copia[columna].str.lower()
        df_copia[columna] = df_copia[columna].str.replace("-", "")
        df_copia[columna] = df_copia[columna].str.translate(
            str.maketrans("", "", "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~")
        )
    
    # Paso 2: Limpiar variaciones con n-gramas
    # ------------------------------------------------------
    # Esta es la parte especifica del algoritmo de n-gram:
    #
    # - Una el texto sin espacios en blanco
    df[columna] = df[columna].str.join("")
    #
    # - Convierta el texto a una lista de n-gramas
    df[columna] = df[columna].map(
        lambda x: [x[t : t + 2] for t in range(len(x))],
    )
    #
    # - Ordene la lista de n-gramas y remueve duplicados
    df[columna] = df[columna].apply(lambda x: sorted(set(x)))
    #
    # - Convierta la lista de ngramas a una cadena
    df[columna] = df[columna].str.join("")
    # ------------------------------------------------------

     # Convierta el texto a una lista de tokens
    df[columna] = df[columna].str.split()

    # Transforme cada palabra con un stemmer de Porter
    stemmer = nltk.PorterStemmer()
    df[columna] = df[columna].apply(lambda x: [stemmer.stem(word) for word in x])

    # Ordene la lista de tokens y remueve duplicados
    df[columna] = df[columna].apply(lambda x: sorted(set(x)))

    # Convierta la lista de tokens a una cadena de texto separada por espacios
    df[columna] = df[columna].str.join(" ")
    

    # Paso 3: Eliminar duplicados
    df_copia = df_copia.drop_duplicates(keep='first')
    
    return df_copia

df_final = limpiar_dataset_completo(df_solicitudes, umbral_similitud=0.85)


print(f"Filas finales: {len(df_final)}")

C:\Users\Gianca\AppData\Local\Temp\ipykernel_23540\1334881564.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columna] = df[columna].str.join("")
C:\Users\Gianca\AppData\Local\Temp\ipykernel_23540\1334881564.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columna] = df[columna].map(


Filas finales: 10497


C:\Users\Gianca\AppData\Local\Temp\ipykernel_23540\1334881564.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columna] = df[columna].apply(lambda x: sorted(set(x)))
C:\Users\Gianca\AppData\Local\Temp\ipykernel_23540\1334881564.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columna] = df[columna].str.join("")
C:\Users\Gianca\AppData\Local\Temp\ipykernel_23540\1334881564.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [221]:
df_solicitudes["sexo"].value_counts().tolist()

[6810, 3688]